In [ ]:
import matplotlib.pyplot as plt

import cv2
import numpy as np
import os
from keras_facenet import FaceNet

# crossPose database 一个人脸图像数据集

共一个文件夹，一共11652个图片。每个图片的命名格式为：
"人名_x.jpg"" 注意：人名中可能也有下划线。例如：Aaron_Peirsol_1.jpg，

其中，数字1代表人脸的正脸像，可以作为标签读取，2及更大值代表图片旋转或者人物侧脸等复杂姿势的人脸。人脸识别模型facenet，可以实现人脸图片到特征向量的转换，
先读取后缀为1的图片保存为特征向量，标签为人名，然后读取后缀为2（不同姿势的图片）生成特征向量，计算和之前图片的特征向量的距离，得到最小值对应的人名，看看是否和该图片相同，相同则预测成功。最后计算准确率

In [ ]:
# 初始化人脸识别模型
poseModel = FaceNet()

# 数据集路径
data_path = './cp-aligned'  # 替换为你的实际数据集路径

# 存储特征向量和标签
features_dict = {}

# 读取后缀为 0001 的图片，生成特征向量并保存
for filename in os.listdir(data_path):
    if filename.endswith("1.jpg"):
        name = filename.rsplit('_',1)[0]  # 提取人名
        img_path = os.path.join(data_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (160, 160),cv2.INTER_LANCZOS4)  # 调整为160x160
        img = np.expand_dims(img, axis=0)  # 添加批次维度
        
        embedding = poseModel.embeddings(img)
        features_dict[name] = embedding  # 用人名作为键

# 现在读取后缀为 0002 的图片并进行相似度比较
correct_predictions = 0
total_images = 0

for filename in os.listdir(data_path):
    if not filename.endswith("1.jpg"):
        name =filename.rsplit('_',1)[0]  # 提取人名
        img_path = os.path.join(data_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (160, 160),cv2.INTER_LANCZOS4)  # 调整为160x160
        img = np.expand_dims(img, axis=0)  # 添加批次维度
        
        embedding = poseModel.embeddings(img)

        # 计算与所有 1 图片的距离
        distances = {}
        for label, feature in features_dict.items():
            distance = np.linalg.norm(embedding - feature)  # 使用 L2 范数计算距离
            distances[label] = distance
        
        # 找到距离最小的人名
        closest_name = min(distances, key=distances.get)

        # 比较是否与当前图片的人名相同
        if closest_name == name:
            correct_predictions += 1
        
        total_images += 1
        

In [ ]:
# 计算准确率
if total_images > 0:
    accuracy = correct_predictions / total_images
    print(f'准确率: {accuracy:.2%}')
else:
    print('没有可测试的图片。')